In [1]:
!git clone https://github.com/deeplearningzerotoall/PyTorch/

Cloning into 'PyTorch'...
remote: Enumerating objects: 1899, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1899 (delta 91), reused 88 (delta 88), pack-reused 1803
Receiving objects: 100% (1899/1899), 80.33 MiB | 36.02 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
trans = transforms.Compose([transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder(root='/content/PyTorch/custom_data/train_data', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [6]:
# model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [7]:
# testing 
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [8]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [9]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.6551658511161804
[Epoch:2] cost = 0.4942258596420288
[Epoch:3] cost = 0.23500534892082214
[Epoch:4] cost = 0.06870826333761215
[Epoch:5] cost = 0.026445452123880386
[Epoch:6] cost = 0.013893328607082367
[Epoch:7] cost = 0.008642658591270447
Learning Finished!


In [18]:
torch.save(net.state_dict(), '/content/PyTorch/model/model.pth')

In [20]:
new_net = CNN().to(device)

In [22]:
new_net.load_state_dict(torch.load('/content/PyTorch/model/model.pth'))

<All keys matched successfully>

In [23]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929],
       grad_fn=<SelectBackward0>)
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929],
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

In [24]:
test_image = '/content/PyTorch/custom_data/test_data'

trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root=test_image, transform=trans)

In [25]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [26]:
# test
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
